# Problem 3

In [174]:
# Create a new model
m = gp.Model("matrix1")
# Set objective
obj = np.asarray([20,10,6,20,10,6,20,10,20])
myvars=['Day 1 Buy', 'Day 1 FC', 'Day 1 SC', 'Day 2 Buy', 'Day 2 FC', 'Day 2 SC', 'Day 3 Buy', 'Day 3 FC', 'Day 4 Buy']
myconst=['Day 1 Demand', 'Day 2 Demand', 'Day 3 Demand','Day 4 Demand', 
         'On-hand >= clean Day 1', 'On-hand >= clean Day 2', 'On-hand >= clean Day 3']
# Create variables
x = m.addMVar(shape=len(obj), vtype=GRB.CONTINUOUS, name="x")
m.setObjective(obj @ x, GRB.MINIMIZE)
A= np.asarray( 
     [[1,0,0,0,0,0,0,0,0],    #Day 1
      [0,1,0,1,0,0,0,0,0],    #Day 2
      [0,0,1,0,1,0,1,0,0],    #Day 3
      [0,0,0,0,0,1,0,1,1],    #Day 4
      [1,-1,-1,0,0,0,0,0,0],  #OH>=Clean Day 1
      [1,0,-1,1,-1,-1,0,0,0], #OH>=Clean Day 2
      [1,0,0,1,0,-1,1,-1,0]   #OH>=Clean Day 3
     ])
# Build rhs vector
b =np.asarray([15,12,18,6,0,0,0])
# Add constraints
m.addConstr(A @ x >= b)
# Optimize model
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 7 rows, 9 columns and 22 nonzeros
Model fingerprint: 0x3c17c256
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 2e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+02   3.600000e+01   0.000000e+00      0s
       6    6.6600000e+02   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds
Optimal objective  6.660000000e+02


In [ ]:
print('Objective Value:', m.objVal)

## Primal

In [172]:
s=x.X
rc=x.RC
ll=x.SAObjLow
ul=x.SAObjUp
mydf=pd.DataFrame({'X':s, 'Reduced_Cost':rc, 'Lower_Limits:':ll, 'Upper_Limits':ul}, index=myvars)
mydf

,X,Reduced_Cost,Lower_Limits:,Upper_Limits
Day 1 Buy,15.0,0.0,10.0,inf
Day 1 FC,9.0,0.0,6.0,12.0
Day 1 SC,6.0,0.0,4.0,10.0
Day 2 Buy,3.0,0.0,18.0,24.0
Day 2 FC,12.0,0.0,6.0,12.0
Day 2 SC,0.0,2.0,4.0,inf
Day 3 Buy,0.0,4.0,16.0,inf
Day 3 FC,6.0,0.0,0.0,12.0
Day 4 Buy,0.0,10.0,10.0,inf


## Dual

In [168]:
duals=m.pi
slack=np.round(m.slack,3)
RHSLL=np.round(m.SARHSLow,3)
RHSUL=np.round(m.SARHSUp,3)
mydf2=pd.DataFrame({'Pi':duals, 'Slack':slack, 'RHS_Lower_Limit:':RHSLL, 'RHS_Upper_Limit':RHSUL}, index=myconst)
mydf2

,Pi,Slack,RHS_Lower_Limit:,RHS_Upper_Limit
Day 1 Demand,10.0,0.0,6.0,18.0
Day 2 Demand,14.0,0.0,3.0,18.0
Day 3 Demand,16.0,0.0,15.0,27.0
Day 4 Demand,10.0,0.0,-0.0,18.0
On-hand >= clean Day 1,4.0,0.0,-12.0,6.0
On-hand >= clean Day 2,6.0,0.0,-3.0,9.0
On-hand >= clean Day 3,0.0,-12.0,-inf,12.0


# Problem 4

In [176]:
# Create a new model
m = gp.Model("matrix1")
# Set objective
obj = np.asarray([9,8,7,10,9,8,9,11,10,7,9,8,9,8,10,9,8,10])
# Create variables
x = m.addMVar(shape=len(obj), vtype=GRB.CONTINUOUS, name="x")
myvars=['X111','X112','X113','X121','X122','X123',
       'X211','X212','X213','X221','X222','X223',
       'X311','X312','X313','X321','X322','X323']
myconst=['P1 4 Courses','P2 4 Courses', 'P3 4 Courses', 
        'Fall Marketing','Fall Finance', 'Fall Production',
        'Spring Marketing','Spring Finance', 'Spring Production',
        'Total Marketing', 'Total Finance', 'Total Production']
m.setObjective(obj @ x, GRB.MAXIMIZE)
A= np.asarray( 
     [[-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0], #Prof1
      [0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0], #Prof2
      [0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1], #Prof3
      [1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0], #Marketing Fall
      [0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0], #Finance Fall
      [0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0], #Production Fall
      [0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0], #Marketing Spring
      [0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0], #Finance Spring
      [0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1], #Production Spring
      [1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0],#Marketing Year
      [0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0],#Finance Year
      [0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1] #Production Year  
     ])
# Build rhs vector
b =np.asarray([-4,-4,-4,1,1,1,1,1,1,4,4,4])
# Add constraints
m.addConstr(A@x>=b)
# Optimize model
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 12 rows, 18 columns and 54 nonzeros
Model fingerprint: 0xddfc0075
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.00s
Presolved: 12 rows, 18 columns, 54 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5900000e+32   1.800000e+31   1.590000e+02      0s
       8    1.2100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds
Optimal objective  1.210000000e+02


In [177]:
print('Objective Value:', m.objVal)

Objective Value: 121.0


# Primal

In [178]:
s=x.X
rc=x.RC
ll=x.SAObjLow
ul=x.SAObjUp
mydf=pd.DataFrame({'X':s, 'Reduced_Cost':rc, 'Lower_Limits:':ll, 'Upper_Limits':ul}, index=myvars)
mydf

,X,Reduced_Cost,Lower_Limits:,Upper_Limits
X111,0.0,0.0,8.0,9.0
X112,0.0,-3.0,-inf,11.0
X113,0.0,-3.0,-inf,10.0
X121,3.0,0.0,9.0,11.0
X122,1.0,0.0,9.0,11.0
X123,0.0,-2.0,-inf,10.0
X211,1.0,0.0,9.0,10.0
X212,3.0,0.0,10.0,inf
X213,0.0,-0.0,-inf,10.0
X221,0.0,-3.0,-inf,10.0


# Duals

In [179]:
duals=m.pi
slack=np.round(m.slack,3)
RHSLL=np.round(m.SARHSLow,3)
RHSUL=np.round(m.SARHSUp,3)
mydf2=pd.DataFrame({'Pi':duals, 'Slack':slack, 'RHS_Lower_Limit:':RHSLL, 'RHS_Upper_Limit':RHSUL}, index=myconst)
mydf2

,Pi,Slack,RHS_Lower_Limit:,RHS_Upper_Limit
P1 4 Courses,-11.0,0.0,-5.0,-4.0
P2 4 Courses,-11.0,0.0,-inf,-4.0
P3 4 Courses,-11.0,0.0,-5.0,-4.0
Fall Marketing,-1.0,0.0,1.0,3.0
Fall Finance,0.0,-2.0,-inf,3.0
Fall Production,-0.0,0.0,-0.0,3.0
Spring Marketing,0.0,-2.0,-inf,3.0
Spring Finance,-2.0,0.0,-0.0,1.0
Spring Production,0.0,-2.0,-inf,3.0
Total Marketing,-1.0,0.0,3.0,4.0
